In [1]:
%cd "C:\Users\user\Documents\UST\BDT\MSBD5005\project\data\"

C:\Users\user\Documents\UST\BDT\MSBD5005\project\data


In [2]:
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
!pip install snscrape
!pip install wheel
!pip install pandas

  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\user\appdata\local\temp\pip-req-build-fkakihti



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pickle
import re
import snscrape
import pandas as pd
import math
from collections import Counter
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [4]:
# file = './tweet_230201-230228.pkl'
file = './tweet_combine_221130-230315.pkl'

with open(file, 'rb') as f:
    tweets_dict = pickle.load(f)

In [5]:
country_list = []
countryCode_list = []
for tid, tweet in tweets_dict.items():
  if tweet.place:
    country_list.append(tweet.place.country)
    countryCode_list.append(tweet.place.countryCode)

country_counter = Counter(country_list)
countryCode_counter = Counter(countryCode_list)
print(country_counter)
print(countryCode_counter)

Counter({'United States': 8679, 'India': 2105, 'United Kingdom': 1745, 'Canada': 909, 'Australia': 658, 'Pakistan': 547, 'Germany': 368, 'Nigeria': 318, 'Kenya': 304, 'Spain': 265, 'United Arab Emirates': 233, 'The Netherlands': 225, 'France': 215, 'South Africa': 211, 'Ireland': 185, 'Malaysia': 170, 'Singapore': 170, 'Italy': 142, 'New Zealand': 129, 'Brazil': 128, 'Portugal': 111, 'Republic of the Philippines': 109, 'Switzerland': 105, 'Indonesia': 95, 'Thailand': 91, 'Israel': 77, 'Japan': 76, "People's Republic of China": 75, 'Belgium': 74, 'Bangladesh': 68, 'Sweden': 65, 'Ghana': 65, 'Finland': 64, 'Mexico': 62, 'Uganda': 61, 'Poland': 60, 'Turkey': 59, 'Norway': 58, 'Argentina': 57, 'Kingdom of Saudi Arabia': 54, 'Tanzania': 47, 'Hong Kong': 44, 'Denmark': 42, 'Sri Lanka': 40, 'Nepal': 37, 'Republic of Korea': 36, 'Republic of Croatia': 32, 'Taiwan': 31, 'Colombia': 30, 'Greece': 30, 'Lebanon': 30, 'Zambia': 27, 'Costa Rica': 27, 'Romania': 24, 'Hungary': 24, 'Malawi': 23, 'Ukra

In [6]:
# remove occurance of r in s, mainly for removing '\n'
def removeAll(s: str, r: str) -> str:
    while r in s:
        s = s.replace(r, ' ')
    return s

In [7]:
#Example of tweets_dict's item'
# {1630357108662075402: Tweet(url='https://twitter.com/MeEmpratoor/status/1630357108662075402', date=datetime.datetime(2023, 2, 27, 23, 59, 59, tzinfo=datetime.timezone.utc), rawContent='@themarcusramsey Learn chatGPT guys for everything.', renderedContent='@themarcusramsey Learn chatGPT guys for everything.', id=1630357108662075402, user=User(username='MeEmpratoor', id=1438758246995611648, displayname='THE YOUNG ENTREPRENEUR', rawDescription='ENTREPRENEUR AND WOLRD WIDE BUSINESSMAN, IS AN UNKNOWN  FUTURE BILLIONAIRE.', renderedDescription='ENTREPRENEUR AND WOLRD WIDE BUSINESSMAN, IS AN UNKNOWN  FUTURE BILLIONAIRE.', descriptionLinks=None, verified=False, created=datetime.datetime(2021, 9, 17, 6, 54, 53, tzinfo=datetime.timezone.utc), followersCount=38, friendsCount=474, statusesCount=23, favouritesCount=55, listedCount=4, mediaCount=2, location='', protected=False, link=None, profileImageUrl='https://pbs.twimg.com/profile_images/1538779124789133313/w8HEw9OJ_normal.jpg', profileBannerUrl='https://pbs.twimg.com/profile_banners/1438758246995611648/1655708645', label=None), replyCount=0, retweetCount=0, likeCount=0, quoteCount=0, conversationId=1630222704191176704, lang='en', source='<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', sourceUrl='http://twitter.com/download/android', sourceLabel='Twitter for Android', links=None, media=None, retweetedTweet=None, quotedTweet=None, inReplyToTweetId=1630222704191176704, inReplyToUser=User(username='themarcusramsey', id=89173194, displayname='Marcus Ramsey', rawDescription=None, renderedDescription=None, descriptionLinks=None, verified=None, created=None, followersCount=None, friendsCount=None, statusesCount=None, favouritesCount=None, listedCount=None, mediaCount=None, location=None, protected=None, link=None, profileImageUrl=None, profileBannerUrl=None, label=None), mentionedUsers=[User(username='themarcusramsey', id=89173194, displayname='Marcus Ramsey', rawDescription=None, renderedDescription=None, descriptionLinks=None, verified=None, created=None, followersCount=None, friendsCount=None, statusesCount=None, favouritesCount=None, listedCount=None, mediaCount=None, location=None, protected=None, link=None, profileImageUrl=None, profileBannerUrl=None, label=None)], coordinates=None, place=None, hashtags=None, cashtags=None, card=None, viewCount=23, vibe=None),

In [16]:
arr = []
df_column = ['tid', 'rawContent', 'renderedContent', 'datetime', 'date', 'longitude', 'latitude', 'country', 'countryCode', 'viewCount', 'viewCount_log', 'likeCount', 'likeCount_log']

for tid, tweet in tweets_dict.items():
  if tweet.coordinates:
    rawContent = removeAll(tweet.rawContent, '\n')
    renderedContent = removeAll(tweet.renderedContent, '\n')
    _datetime = tweet.date
    _date = _datetime.date()
    longitude = tweet.coordinates.longitude
    latitude = tweet.coordinates.latitude
    country = tweet.place.country
    countryCode = tweet.place.countryCode
    viewCount = tweet.viewCount if tweet.viewCount else 0
    viewCount_log = math.log(viewCount+1)
    likeCount = tweet.likeCount if tweet.likeCount else 0
    likeCount_log = math.log(likeCount+1)
    arr.append((tid, rawContent, renderedContent, _datetime, _date, longitude, latitude, country, countryCode, viewCount, viewCount_log, likeCount, likeCount_log))

df = pd.DataFrame(arr, columns=[df_column])


In [18]:
df.head()

,tid,rawContent,renderedContent,datetime,date,longitude,latitude,country,countryCode,viewCount,viewCount_log,likeCount,likeCount_log
0,1631079802902835200,"So much potential with the new @OpenAI ChatGPT API, and significantly cheaper than their existing options","So much potential with the new @OpenAI ChatGPT API, and significantly cheaper than their existing options",2023-03-01 23:51:43+00:00,2023-03-01,-74.041878,40.570842,United States,US,299,5.703782,1,0.693147
1,1631078755950292992,"@MerlinCrossley Absolutely - and everyone will have ChatGPT, so what will grads need to give them an edge? Creativity, and that human ability to not think like a machine, will remain the huge difference.","@MerlinCrossley Absolutely - and everyone will have ChatGPT, so what will grads need to give them an edge? Creativity, and that human ability to not think like a machine, will remain the huge difference.",2023-03-01 23:47:34+00:00,2023-03-01,144.593742,-38.433859,Australia,AU,48,3.891820,3,1.386294
2,1631077423482798081,"ChatGPT API is officially published, cheers!","ChatGPT API is officially published, cheers!",2023-03-01 23:42:16+00:00,2023-03-01,-121.997026,37.205924,United States,US,326,5.789960,1,0.693147
3,1631072894112628737,Good point but graduates will see times when they can and moments where they can’t access ChatGPT so they need to be skilled in its use and in being independent of it (thinking on their feet),Good point but graduates will see times when they can and moments where they can’t access ChatGPT so they need to be skilled in its use and in being independent of it (thinking on their feet),2023-03-01 23:24:16+00:00,2023-03-01,150.520929,-34.118347,Australia,AU,650,6.478510,4,1.609438
4,1631072767071199232,@iborganization addressing the ways in which assessment will undoubtedly evolve as technology does. #ChatGPT #schools #education #essaywriting https://t.co/LlFrQO7lpx,@iborganization addressing the ways in which assessment will undoubtedly evolve as technology does. #ChatGPT #schools #education #essaywriting amp-theguardian-com.cdn.ampproject.org/c/s/amp.thegua…,2023-03-01 23:23:46+00:00,2023-03-01,-2.543152,54.318511,United Kingdom,GB,9,2.302585,0,0.000000


In [20]:
df.to_csv(re.sub('.pkl$', '.csv', file))